<a href="https://colab.research.google.com/github/immprasetyo/ibm-data-science/blob/main/folium-dash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# **Launch Sites Locations Analysis with Folium**


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [ ]:
#!pip freeze

In [ ]:
import pandas as pd
import requests
import folium

In [ ]:
from folium.features import DivIcon
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module\_3/DV0101EN-3-5-1-Generating-Maps-in-Python-py-v2.0.ipynb)


### Task 1: Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [ ]:
# Download and read the `spacex_launch_geo.csv`
f_url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = requests.get(f_url)
content = resp.content

In [ ]:
with open('spacex_launch_geo.csv', 'wb') as f:
  f.write(content)

spacex_df = pd.read_csv('spacex_launch_geo.csv')
spacex_df.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


In [ ]:
spacex_df.rename(columns = {'class': 'Class'}, inplace = True)

In [ ]:
spacex_df.shape

(56, 13)

Now, you can take a look at what are the coordinates for each site.


In [ ]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'Class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index = False).first()
launch_sites_df

,Launch Site,Lat,Long,Class
0,CCAFS LC-40,28.562302,-80.577356,0
1,CCAFS SLC-40,28.563197,-80.576820,1
2,KSC LC-39A,28.573255,-80.646895,1
3,VAFB SLC-4E,34.632834,-120.610745,0


In [ ]:
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [ ]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.55968, -95.08310]  #[29.559684888503615, -95.0830971930759]
site_map = folium.Map(location = nasa_coordinate, zoom_start = 10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [ ]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
nasa_circle = folium.Circle(
    nasa_coordinate,
    radius = 1000,
    color = 'royalblue',
    fill = True
).add_child(folium.Popup('NASA Johnson Space Center'))

# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
nasa_marker = folium.map.Marker(
    nasa_coordinate,
    
    # Create an icon as a text label
    icon = DivIcon(
        icon_size = (20, 20),
        icon_anchor = (0, 0),
        html = "<div style = 'font-size: 12; color: royalblue'><b>%s</b></div>" % 'NASA JSC'
    )
)

site_map.add_child(nasa_circle)
site_map.add_child(nasa_marker)

In [ ]:
#site_map = folium.Map()
#site_map

In [ ]:
#site_map._children

OrderedDict([('openstreetmap', <folium.raster_layers.TileLayer at 0x5cbc7c0>),
             ('marker_cluster_a7d61467c70259c72ee2764feb09e7e3',
              <folium.plugins.marker_cluster.MarkerCluster at 0x4773290>),
             ('marker_cluster_9664558c366c4ecfea5b23c6c9c998c8',
              <folium.plugins.marker_cluster.MarkerCluster at 0x5b2c410>)])

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [ ]:
# Initial the map
site_map = folium.Map(location = nasa_coordinate, zoom_start = 5)

# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
for i, row in launch_sites_df.iterrows():
    coordinate = [row['Lat'], row['Long']]
    
    folium.Circle(
        coordinate,
        radius = 1000,
        color = 'indianred',
        fill = True
    ).add_child(folium.Popup(row['Launch Site'])).add_to(site_map)
    
    folium.map.Marker(
        coordinate,
        icon = DivIcon(
            icon_size = (20, 20),
            icon_anchor = (0, 0),
            html = "<div style = 'font-size: 12; color: indianred'><b>%s</b></div>" % row['Launch Site']
        )
    ).add_to(site_map)

site_map

The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


##### Findings:
*  All launch sites are in proximity to the Equator Line and the coast. This because it takes less fuel to get into space from Equator Line. Also it is safer to launch rocket from coast since it is far from densely populated residential areas

### Task 2: Mark the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [ ]:
spacex_df.tail(10)

,Launch Site,Lat,Long,Class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [ ]:
marker_cluster = MarkerCluster()  # Useful for resetting

*TODO:* Create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value


In [ ]:
# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'darkseagreen'
    else:
        return 'indianred'

In [ ]:
spacex_df['Marker Color'] = spacex_df['Class'].apply(assign_marker_color)
spacex_df.tail(10)

,Launch Site,Lat,Long,Class,Marker Color
46,KSC LC-39A,28.573255,-80.646895,1,darkseagreen
47,KSC LC-39A,28.573255,-80.646895,1,darkseagreen
48,KSC LC-39A,28.573255,-80.646895,1,darkseagreen
49,CCAFS SLC-40,28.563197,-80.576820,1,darkseagreen
50,CCAFS SLC-40,28.563197,-80.576820,1,darkseagreen
51,CCAFS SLC-40,28.563197,-80.576820,0,indianred
52,CCAFS SLC-40,28.563197,-80.576820,0,indianred
53,CCAFS SLC-40,28.563197,-80.576820,0,indianred
54,CCAFS SLC-40,28.563197,-80.576820,1,darkseagreen
55,CCAFS SLC-40,28.563197,-80.576820,0,indianred


*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [ ]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# For each row in spacex_df data frame create a Marker object with its coordinate and customize the Marker's icon property to indicate if this launch was
# successed or failed, e.g. icon=folium.Icon(color='white', icon_color=row['marker_color']
for i, row in spacex_df.iterrows():
    
    # TODO: Create and add a Marker cluster to the site map
    marker = folium.Marker(
        [row['Lat'], row['Long']],
        icon = folium.Icon(
            color = 'white',
            icon_color = row['Marker Color']
        )
    )

    marker_cluster.add_child(marker)

site_map

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


### TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [ ]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = 'function(num){return L.Util.formatNum(num, 5)}'

mouse_position = MousePosition(
    position = 'topright',
    separator = ' Long: ',
    empty_string = 'NaN',
    lng_first = False,
    num_digits = 20,
    prefix = 'Lat: ',
    lat_formatter = formatter,
    lng_formatter = formatter
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [ ]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    
    # Approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)    

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = sin(dlat/2)**2 + cos(lat1)*cos(lat2)*sin(dlon/2)**2
    c = 2*atan2(sqrt(a), sqrt(1 - a))

    distance = R*c
    
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [ ]:
# Find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
launch_site_lat = 28.56275
launch_site_lon = -80.57705
coastline_lat = 28.56195
coastline_lon = -80.56780

coastline_distance = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

In [ ]:
print('Coatline distance from launch site: %.3f km' % coastline_distance)

Coatline distance from launch site: 0.908 km


In [ ]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
coordinate = [coastline_lat, coastline_lon]

distance_marker = folium.Marker(
    coordinate,
    icon = DivIcon(
        icon_size = (20, 20),
        icon_anchor = (0, 0),
        html = "<div style = '<div style = 'font-size: 12; color: royalblue'><b>%s</b></div>" % '{:5.2f} km'.format(coastline_distance)
    )
)

site_map.add_child(distance_marker)

*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [ ]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# lines=folium.PolyLine(locations=coordinates, weight=1)
coordinates = [[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]]
lines = folium.PolyLine(locations = coordinates, weight = 1)
site_map.add_child(lines)

Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [ ]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site
closest_city =  [28.10471, -80.64533]
closest_highway = [28.56333, -80.57087]
closest_railroad = [28.57205, -80.58525]

In [ ]:
city_distance = calculate_distance(launch_site_lat, launch_site_lon, closest_city[0], closest_city[1])
highway_distance = calculate_distance(launch_site_lat, launch_site_lon, closest_highway[0], closest_highway[1])
railroad_distance = calculate_distance(launch_site_lat, launch_site_lon, closest_railroad[0], closest_railroad[1])

print('City distance: {:10.3f} km\nHighway distance: {:6.3f} km\nRailroad distance: {:3.3f} km'.format(city_distance, highway_distance, railroad_distance))

City distance:     51.384 km
Highway distance:  0.607 km
Railroad distance: 1.308 km


In [ ]:
# Closest city marker
distance_marker = folium.Marker(
    closest_city,
    icon = DivIcon(
        icon_size = (20, 20),
        icon_anchor = (0, 0),
        html = "<div style = '<div style = 'font-size: 12; color: royalblue'><b>%s</b></div>" % '{:10.2f} km'.format(city_distance)
    )
)

site_map.add_child(distance_marker)

# Closest city line
coordinates = [[launch_site_lat, launch_site_lon], closest_city]
lines = folium.PolyLine(locations = coordinates, weight = 1)
site_map.add_child(lines)



# Closest highway marker
distance_marker = folium.Marker(
    closest_highway,
    icon = DivIcon(
        icon_size = (20, 20),
        icon_anchor = (0, 0),
        html = "<div style = '<div style = 'font-size: 12; color: royalblue'><b>%s</b></div>" % '{:10.2f} km'.format(highway_distance)
    )
)

site_map.add_child(distance_marker)

# Closest highway line
coordinates = [[launch_site_lat, launch_site_lon], closest_highway]
lines = folium.PolyLine(locations = coordinates, weight = 1)
site_map.add_child(lines)



# Closest railroad marker
distance_marker = folium.Marker(
    closest_railroad,
    icon = DivIcon(
        icon_size = (20, 20),
        icon_anchor = (0, 0),
        html = "<div style = 'font-size: 12; color: royalblue;'><b>%s</b></div>" % '{:10.2f} km'.format(railroad_distance)
    )
)

site_map.add_child(distance_marker)

# Closest railroad line
coordinates = [[launch_site_lat, launch_site_lon], closest_railroad]
lines = folium.PolyLine(locations = coordinates, weight = 1)
site_map.add_child(lines)

After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


##### Findings:
* Launch sites are in close proximity to railways which allows transport for logistic and heavy cargo
* Launch sites are in close proximity to highways which allows transport for crew and employee
* Launch sites are in close proximity to coastline which allows crew to abort launch and attempt water landing
* Launch sites are certain distance away from cities which minimizes danger to densely populated residential area

# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork865-2022-01-01)


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
